# SimpleBaseline - tf2

- 파이토치 코드는 케라스 보다 직관적으로 표현되어 있기 때문에 지금처럼 빠르게 파악할 수 있는 장점이 있습니다.
- 이제 파악한 지식을 이용해서 tf-simplebaseline 모델을 만들어봅시다.

In [1]:
#import os

#import numpy as np
#import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

deconv module을 만들어주세요.

In [2]:
upconv1 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn1 = tf.keras.layers.BatchNormalization()
relu1 = tf.keras.layers.ReLU()
upconv2 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn2 = tf.keras.layers.BatchNormalization()
relu2 = tf.keras.layers.ReLU()
upconv3 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn3 = tf.keras.layers.BatchNormalization()
relu3 = tf.keras.layers.ReLU()

deconv module에서 중복을 제거할 수 없을까요? 코드로 표현해 봅시다.

In [3]:
def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

upconv = _make_deconv_layer(3)

In [4]:
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')

이제 각각의 요소를 합쳐 모델을 완성합니다.

In [5]:
inputs = keras.Input(shape=(256, 192, 3))
x = resnet(inputs)
x = upconv(x)
out = final_layer(x)
model = keras.Model(inputs, out)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 192, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
sequential (Sequential)      (None, 64, 48, 256)       10489600  
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 48, 17)        4369      
Total params: 34,081,681
Trainable params: 34,027,025
Non-trainable params: 54,656
_________________________________________________________________


가상의 이미지를 넣어서 출력이 잘 나오는지 확인하는 코드를 작성해 봅시다.

In [6]:
np_input = np.zeros((1,256,192,3), dtype=np.float32)
tf_input = tf.convert_to_tensor(np_input, dtype=tf.float32)
print('input shape')
print (tf_input.shape)
print('\n')

tf_output = model(tf_input)
print('output shape')
print (tf_output.shape)
print (tf_output[0,:10,:10,:10])

input shape
(1, 256, 192, 3)


output shape
(1, 64, 48, 17)
tf.Tensor(
[[[-1.45826407e-03 -4.51131165e-03 -3.81750893e-03  2.29676021e-03
    9.14883974e-04 -6.00711210e-03  8.85433424e-03  3.52671690e-04
   -6.34759897e-04  1.11285492e-03]
  [-1.82177068e-03 -2.08216272e-02  9.85977938e-04 -6.69232337e-03
    2.63027905e-04 -7.56258611e-03 -5.38755034e-04 -6.03525247e-03
    1.58477519e-02 -4.20104858e-04]
  [-6.15824945e-03 -1.65514043e-03 -5.29490225e-03  7.58959167e-03
    1.00536095e-02 -9.11986548e-03  1.57882608e-02 -3.26515851e-03
    9.65116452e-03  2.31056660e-03]
  [ 8.58976319e-03  2.17779074e-02  6.54118555e-03  5.87506685e-03
    2.34982511e-03 -1.83072072e-02  1.34124095e-03 -3.49727534e-02
    5.67997154e-03 -1.73948873e-02]
  [-2.34282408e-02  8.03265534e-03 -1.97132918e-04 -8.28289334e-03
    1.20806713e-02 -1.60361230e-02  2.42705066e-02  9.58856661e-04
    2.65834248e-03 -1.25973085e-02]
  [-6.01731997e-04  6.66403351e-03 -1.35310832e-02  5.16459113e-03
    7.320328